# Betalyzer


## Installation

I'm running Anaconda with the version dump below. I've tested this code using the latest builds of most libraries. The only library that I'm using that does not come default with Anaconda is Quandl, which would need to be installed using `pip install Quandl`. Note that unless you Recalculate (where Quandl is necessary), you don't need to install Quandl to run this application.

In [1]:
import sys
sys.version

'3.5.2 |Anaconda custom (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]'

## Technologies
I used the following components (other than the obvious like Python, HTML, CSS, jQuery).
 - **[Quandl](http://www.quandl.com):** I chose Quandl as the primary datasource because the API is documented and the data sources are much fuller than Yahoo Finance.
 - **[Pandas](http://pandas.pydata.org/):** Most calculations are done in pandas. Pandas uses numpy in the backend, but provides a layer of abstraction as well as a variety of tools built on top. 
 - **[Flask](http://flask.pocoo.org/):** Flask is a lightweight web framework written in Python. As they say, the Navy uses Django; pirates use Flask. 
 - **[Bokeh](http://bokeh.pydata.org/en/latest/):** Though I've used primarily Highcharts and D3 before, this is my first project using bokeh for data visualization. I'm extremely impressed by the ease of data viz that it provides as well as its seamless integration with both web output as well as pandas DataFrame inputs.
 - **[Bootstrap](http://getbootstrap.com/):** I use Twitter's Bootstrap v3 on the frontend.
 - **[Datatables](https://datatables.net/):** Datatables is a jQuery plugin that renders tables for the web extremely quickly. It has a lot of functionality around sorting, viewing, etc.
 - **[Jupyter Notebook](http://jupyter.org/):** I do most of my prototyping in Jupyter, as well as use it to write these reports.

## Application Structure
Betalyzer is a Flask app but there are several files.
 - **`betalyzer.py`:** All of the business logic is in this file.
 - **`app.py`:** This is the Flask app and handles the execution of the web app.
 - **`templates/`:** We have three pages -- `index.html` which has our app's outline, `main.html` which displays the content on the main page (the Datatable and the charts) and `ticker.html` which shows the timeseries of beta for a single ticker.
 - **Pickles:** For performance reasons, I do not want to fetch the data and recalculate the historical betas every time a user opens the web page. So I store the calculations in pickle files, including `df_tickers` for all the tickers, `df_betas` as a timeseries for historical betas by ticker and `df_changes` as a timeseries for daily changes by ticker.
 - **`betalyzer.ipynb`:** This file. This is the resulting report.
 - **`beta-calc-optimizations.ipynb`:** This shows the optimizations performed to improve the calculation speed by a factor of 3000x.
 - **`play.ipynb`:** I have left the play Notebooks in place where I did some of my sratch work, though none of them are actually used.

## Data Issues
There were a variety of data issues that I came across. 

First, getting a list of current tickers was a bit tricky. Neither Quandl nor Yahoo Finance provided an easy way to accomplish this. However, NASDAQ publishes a daily list of securities traded on its exchange. `nasdaq_url` in `betalyzer.py` sets this source. So I only have NASDAQ stocks in Betalyzer currently.

Second, according to Quandl, their best data source for daily stock price data is WIKI. However, WIKI does not have ETFs, and to calculate the market move, I would like to use an ETF such as SPY. Therefore, I use WIKI for all individual stock price data, and I use GOOG for SPY data.

Third, GOOG is not perfect. It does not seem to have a very good calendar in the API, and contains non business days (primarily holidays as opposed to weekends). Also, GOOG does not adjust for stock splits and dividends. Since we only get SPY from GOOG, the main issue is that it is not dividend adjusted. Meanwhile, WIKI has adjusted for dividends and splits as we use the column 'Adj Close'.

Fourth, there is missing data in WIKI from Quandl. For example, GOOG only starts in mid 2014. Also, a ticker like 'CELG' (randomly chosen) has several missing data points:

```
[In] df_changes['CELG'][df_changes['CELG'].isnull()]
[Out]
Date
2014-10-02   NaN
2014-11-05   NaN
2014-12-03   NaN
2014-12-08   NaN
2014-12-11   NaN
2014-12-29   NaN
2015-01-07   NaN
2015-01-09   NaN
2015-01-16   NaN
2015-01-27   NaN
2015-02-05   NaN
2015-02-06   NaN
2015-07-13   NaN
Name: CELG, dtype: float64
```

We could use .fillna(0) to solve this issue temporarily, but I have not done this, so missing data will appear as gaps in the beta calculation charts.

## Extensions
Should this have been a production level project, here are some extensions that could have been added:
 - **Additional data:** We currently only get NASDAQ data. We should get data from NYSE as well to cover most equities. We also do not get ETFs. Additionally, there are some shortcomings in the adjustments to the SPY data (it is not total return) that need to be taken into account. 
 - **Datastore:** The data is currently stored in pickled DataFrames, and only the data we need (the rest is discarded). Ideally, we'd move this to a Postgres database, and perhaps Redshift if the data starts moving beyond a few GBs, for optimal performance. 
 - **Recalculation:** Currently, on recalculation, all data is fetched from Quandl, even data that we've used before, and all calculations are made again. Ideally, we should only fetch new data and calculate based on the new data. 
 - **Auto-recalculation:** Currently, only manual recalculation is supported. I would add a process that auto-recalculates every business day at a certain time when both NASDAQ and Quandl have updated their data. Perhaps an asynchronus process and a scheduler (Celery and Celery Beat is what I've used in the past) could help here. 